In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import BertModel
from datasets import load_dataset

from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm

from datasets import load_metric

class PlaceHolderBERT(nn.Module):
    def __init__(self, num_out=1, sigmoid=False, return_CLS_representation=False):
        super().__init__()
        #self.tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.linear = nn.Linear(768,num_out)
        self.return_CLS_representation = return_CLS_representation
        self.sigmoid_bool = sigmoid
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        #embeddings = self.tokenizer(x, return_tensors='pt', padding=True)
        #embeddings.to(device)
        representations = self.bert(**x).last_hidden_state
        cls_representation = representations[:,0,:]
        pred = self.linear(cls_representation)
        if self.return_CLS_representation:
            return cls_representation
        if self.sigmoid_bool:
            return self.sigmoid(pred)
        return pred
    
    
def train(model, dataloader, num_epochs=1): #can scrap keyword
    #optimizer as usual
    optimizer = AdamW(model.parameters(), lr=5e-5)
    loss_function = torch.nn.MSELoss()
    #learning rate scheduler
    num_training_steps = num_epochs * len(dataloader)
    lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    #auto logging; progress bar
    progress_bar = tqdm(range(num_training_steps))

    #training loop
    model.train()
    for epoch in range(num_epochs):
        for batch in dataloader: #tryin unpacking text from 'labels' as in model development
            batch = {k: v.to(device) for k, v in batch.items()}
            features = {k: v for k, v in batch.items() if k != 'labels'}
            preds = model(features)
            loss = loss_function(preds, batch['labels'].float()) #replace .loss
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            

def evaluate(model, dataloader):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)
    model.eval()
    num_correct = 0
    num_samples = 0
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        features = {k: v for k, v in batch.items() if k != 'labels'}
        with torch.no_grad():
            preds = model(features)
            preds = torch.where(preds < .5, 0, 1)
            labels = batch['labels'].reshape(preds.shape)
            num_correct += (preds==labels).sum()
            num_samples += preds.size(0)
    return float(num_correct)/float(num_samples)*100 




imdb = load_dataset('imdb')
sst2 = load_dataset('glue','sst2')

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

#tokenize function
def tokenize_imdb(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

def tokenize_sst2(examples):
    return tokenizer(examples['sentence'], padding="max_length", truncation=True)

#pre-tokenize entire dataset
tokenized_imdb = imdb.map(tokenize_imdb, batched=True)
tokenized_sst2 = sst2.map(tokenize_sst2, batched=True)

tokenized_imdb = tokenized_imdb.remove_columns(["text"])
tokenized_imdb = tokenized_imdb.rename_column("label", "labels")
tokenized_imdb.set_format("torch")

tokenized_sst2 = tokenized_sst2.remove_columns(["sentence","idx"])
tokenized_sst2 = tokenized_sst2.rename_column("label", "labels")
tokenized_sst2.set_format("torch")


### Only for practive
imdb_small_train = tokenized_imdb['train'].shuffle(seed=42).select(range(1000))
imdb_small_test = tokenized_imdb['test'].shuffle(seed=42).select(range(500))
###
imdb_train_loader = DataLoader(imdb_small_train, shuffle=True, batch_size=8)
imdb_test_loader = DataLoader(imdb_small_test, shuffle=True, batch_size=8)

sst2_small_train = tokenized_sst2["train"].shuffle(seed=42).select(range(1000))
sst2_small_test = tokenized_sst2["validation"].shuffle(seed=42).select(range(500)) #actual test set is fucked up

sst2_train_loader = DataLoader(sst2_small_train, shuffle=True, batch_size=8)
sst2_test_loader = DataLoader(sst2_small_test, shuffle=True, batch_size=8)

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import BertModel
from datasets import load_dataset

from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm

from datasets import load_metric

In [4]:


stsb = load_dataset('glue','stsb')
stsb

Reusing dataset glue (/home/ubuntu/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 5749
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1379
    })
})

In [15]:
import csv
data_path = '/home/ubuntu/NLP-brain-biased-robustness/data/stsb/stsbenchmark/'

#wget https://data.deepai.org/Stsbenchmark.zip

def read_csv(csv_file):
    file = open(csv_file)
    csvreader = csv.reader(file, delimiter="\t")
    header = next(csvreader)
    rows = []
    for row in csvreader:
        rows.append(row)
    file.close()
    return rows

In [17]:
train_set = read_csv(data_path+'sts-train.csv')
dev_set = read_csv(data_path+'sts-dev.csv')
test_set = read_csv(data_path+'sts-test.csv')

In [33]:
def split_data():
    headlines = []
    images = []
    MSRpar = []
    MSRvid = []
    for dataset in [train_set, dev_set, test_set]:
        for i in range(len(dataset)):
            if dataset[i][1] == 'headlines':
                headlines.append(dataset[i])
            if dataset[i][1] == 'images':
                images.append(dataset[i])
            if dataset[i][1] == 'MSRpar':
                MSRpar.append(dataset[i])
            if dataset[i][1] == 'MSRvid':
                MSRvid.append(dataset[i])
    return headlines, images, MSRpar, MSRvid

In [34]:
headlines, images, MSRpar, MSRvid = split_data()

In [35]:
headlines

[['main-news',
  'headlines',
  '2013',
  '0000',
  '2.6',
  'Drug lord captured by marines in Mexico',
  'Suspected drug lord known as ‘El Taliban’ held in Mexico'],
 ['main-news',
  'headlines',
  '2013',
  '0001',
  '4.4',
  "SC dismisses govt's review plea in Vodafone tax case",
  "SC dismisses govt's review petition on Vodafone tax verdict"],
 ['main-news',
  'headlines',
  '2013',
  '0002',
  '2.6',
  "Explosion hits oil pipeline in Syria's Homs",
  'Explosion hits pipeline as Assad attacks cities'],
 ['main-news',
  'headlines',
  '2013',
  '0003',
  '3.8',
  'NATO Soldier Killed In Afghan Attack',
  'NATO soldier killed in S. Afghanistan'],
 ['main-news',
  'headlines',
  '2013',
  '0004',
  '4.2',
  'Castro celebrates 86th birthday Monday',
  'Fidel Castro celebrates 86th birthday'],
 ['main-news',
  'headlines',
  '2013',
  '0005',
  '3',
  "Israel ex-spy warns against 'messianic' Iran war",
  'Israeli ex-spy boss: no trust in leadership over Iran – video'],
 ['main-news',
  